<center><br><br>
    Arkansas Work-Based Learning to Workforce Outcomes <br>
    Applied Data Analytics Training | Spring 2022
    <h1> Longitudinal Analysis Interactive Guide </h1>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Coleridge Initiative</a>
    </span>
    <center>Robert McGough, Nishav Mainali, Josh Edelmann, Benjamin Feder</center>
</center>

***

This interactive guide serves as an overview of how a longitudinal analysis can be structured based on the dimension model generated from the code introduced in `03A_Linked_Dataset_Construction.ipynb`. It will also provide you with template queries you can modify to support your project analysis.

At each stage you will be modifying the code with the appropriate variable, function or SQL/R code snippet for your cohort and project goals. 

Through this interactive guide, you will work through code:
1. Reading your cohort constructed in `02_Creating_a_cohort_checkpoints.ipynb` and summarizing cohort characteristics from the resulting linked dataset in a table.
1. Analyzing metrics trends across multiple quarters pre- and post-completion for a cohort of interest and displaying them in a line chart.
1. Comparing metrics between two periods (completion and n quarters post completion) for a cohort of interest and displaying them in a bar chart.

The SQL scripts for all three stages can be found in the Longitudinal Analysis Interactive Guide Queries subfolder.  You can copy these to your personal or team area for modification and viewing results in DBeaver.

**The purpose of this interactive guide is similar to that of previous checkpoint notebooks - we hope you to practice making changes to the code that will help construct your own research project. We have given examples, but these are more applicable to a generic research project.**

In [ ]:
# Switching off warnings
options(warn = -1)

# Database interaction imports
suppressMessages(library(odbc))

# for data manipulation/visualization
suppressMessages(library(tidyverse))

# scaling data, calculating percentages, overriding default graphing
suppressMessages(library(scales))

#Switching on warnings
options(warn = 0)

In [ ]:
# Connect to the database
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

## Stage 1:  Summarizing cohort characteristics from a linked data asset

To start off, we will combine the cohort that you created in **02_Creating_a_cohort_checkpoints.ipynb** with the linked dataset constructed in **03A_Linked_Dataset_Construction.ipynb** and summarize the cohort composition in a table.

This can be used to support a variety of purposes, including:
- Understanding if the subpopulations in a selected cohort are large enough to facilitate robust analysis.
- Understanding if there is signficant missingness in variables of interest that could adversely affect the intended analyses.
- Checking that the cell sizes for the intended analyses are large enough to pass primary statisticial cell size suppression without substantially impacting reporting of the analysis and findings (more on this in future materials).
- Producing a cohort summary table for a project presentation and report.

To complete this stage and use this query:
1. Replace the **INSERT_TABLE_NAME** table name in the flower box with the name of the cohort table that you created in `02_Creating_a_cohort_checkpoints.ipynb`.  You can alternately identify your cohort by adding `WHERE` clauses to the **AR_MDIM_Person table**.
2. Analyze the results in the table below and identify if small cells sizes or missing data (NA) may significantly impact your analyses.
3. Comment or uncomment out additional columns (in both the `SELECT` and the `GROUP BY`) to slice your cohort by different variables at different grains.
4. Try different cohort sizes as well to hone in on a cohort large enough to facilitate robust analysis and dissemination on your variables of interest.

This query also has a `HAVING` clause that is commented out which restricts the values returned to cell sizes larger than 10.  You can uncomment this when you are ready to export results with primary statistical cell size suppression or reverse the sign to understand what will be excluded by suppression. Again, there will be more information provided on exporting results in the ensuing lectures and notebooks.

In [ ]:
# summarize cohort counts and look for missingness amongst variables of interest
cohortqry <- "SELECT
COUNT(P.Person_ID) AS All_Cohort_Members

--PERSON DEMOGRAPHICS
,P.Gender
,P.Race
,P.Ethnicity
,P.Disabled
,P.Veteran

/*
--PUT APPRENTICESHIP STARTING AGE INTO STANDARD DEPT OF LABOR AGE RANGES FOR FEDERAL REPORTING
,CASE WHEN P.Apprenticeship_Age_at_Start < 16 THEN '< 16'	
	WHEN P.Apprenticeship_Age_at_Start BETWEEN 16 AND 18 THEN '16 - 18'
	WHEN P.Apprenticeship_Age_at_Start BETWEEN 19 AND 24 THEN '19 - 24'
	WHEN P.Apprenticeship_Age_at_Start BETWEEN 25 AND 44 THEN '25 - 44'
	WHEN P.Apprenticeship_Age_at_Start BETWEEN 45 AND 54 THEN '45 - 54'
	WHEN P.Apprenticeship_Age_at_Start BETWEEN 55 AND 59 THEN '55 - 59'
	WHEN P.Apprenticeship_Age_at_Start > 60 THEN '60+'
	ELSE NULL
END AS 'Apprenticeship_Age_Range_at_Start'
*/

--INDUSTRY LISTED COURSE TO FINE
--,NNI.NAICS_Supersector_Name
--,NNI.NAICS_Sector_Name
--,NNI.NAICS_Subsector_Name
--,NNI.NAICS_Industry_Group_Name
--,NNI.NAICS_Industry_Name
--,NNI.NAICS_National_Industry_Name

--OCCUPATION
--,S.SOC_Major_Group_Name
--,S.SOC_Minor_Group_Name
--,S.SOC_Broad_Group_Name
--,S.SOC_Detailed_Occupation_Name

--COUNTY
--,CTY.Rural_Urban_Continuum_Name
--,CTY.Local_Workforce_Development_Area
--,CTY.County_Name

FROM 
--*******************************************************************************************************
--ADD YOUR TEAM'S COHORT TABLE HERE
tr_ar_2022.dbo.INSERT_TABLE_NAME C --COHORT
--AN INNER JOIN ON THE COHORT TABLE LIMITS THE SELECTION OF PERSON RECORDS TO THE COHORT OF INTEREST
--*******************************************************************************************************

JOIN tr_ar_2022.dbo.AR_MDIM_Person P ON (P.Apprentice_Number=C.apprnumber) --PERSON

--LOOK UP APPRENTICESHIP INDUSTRY WHERE APPLICABLE.  OUTER (LEFT) JOIN IS USED TO AVOID EXCLUDING ROWS WITHOUT INDUSTRY.
LEFT JOIN tr_ar_2022.dbo.AR_RDIM_NAICS_National_Industry NNI ON (NNI.NAICS_National_Industry_ID=P.Apprenticeship_NAICS_National_Industry_ID)

--LOOK UP APPRENTICESHIP OCCUPATION WHERE APPLICABLE.  OUTER JOIN IS USED TO AVOID EXCLUDING ROWS WITHOUT OCCUPATION.
LEFT JOIN tr_ar_2022.dbo.AR_RDIM_SOC_Detailed_Occupation S ON (S.SOC_Detailed_Occupation_ID=P.Apprenticeship_SOC_Detailed_Occupation_ID)

--LOOK UP APPRENTICESHIP PROGRAM COUNTY WHERE APPLICABLE.  OUTER JOIN IS USED TO AVOID EXCLUDING ROWS WITHOUT COUNTY.
LEFT JOIN tr_ar_2022.dbo.AR_RDIM_County CTY ON (CTY.County_ID=P.Apprenticeship_County_ID)

GROUP BY
--PERSON DEMOGRAPHICS
P.Gender
,P.Race
,P.Ethnicity
,P.Disabled
,P.Veteran

/*
--PUT APPRENTICESHIP STARTING AGE INTO STANDARD DEPT OF LABOR AGE RANGES FOR FEDERAL REPORTING
,CASE WHEN P.Apprenticeship_Age_at_Start < 16 THEN '< 16'	
	WHEN P.Apprenticeship_Age_at_Start BETWEEN 16 AND 18 THEN '16 - 18'
	WHEN P.Apprenticeship_Age_at_Start BETWEEN 19 AND 24 THEN '19 - 24'
	WHEN P.Apprenticeship_Age_at_Start BETWEEN 25 AND 44 THEN '25 - 44'
	WHEN P.Apprenticeship_Age_at_Start BETWEEN 45 AND 54 THEN '45 - 54'
	WHEN P.Apprenticeship_Age_at_Start BETWEEN 55 AND 59 THEN '55 - 59'
	WHEN P.Apprenticeship_Age_at_Start > 60 THEN '60+'
	ELSE NULL
END
*/

--INDUSTRY LISTED COURSE TO FINE
--,NNI.NAICS_Supersector_Name
--,NNI.NAICS_Sector_Name
--,NNI.NAICS_Subsector_Name
--,NNI.NAICS_Industry_Group_Name
--,NNI.NAICS_Industry_Name
--,NNI.NAICS_National_Industry_Name

--OCCUPATION
--,S.SOC_Major_Group_Name
--,S.SOC_Minor_Group_Name
--,S.SOC_Broad_Group_Name
--,S.SOC_Detailed_Occupation_Name

--COUNTY
--,CTY.Rural_Urban_Continuum_Name
--,CTY.Local_Workforce_Development_Area
--,CTY.County_Name

--DATA QUALITY
--,P.Apprenticeship_SSN_Complete


--HAVING COUNT(P.Person_ID) > 10  --UNCOMMENT THIS WHEN YOU ARE READY TO APPLY STATISTICAL CELL SIZE SUPPRESION.  YOU CAN ALSO FLIP THE RELATIONSHIP TO IDENTIFY WHAT IS BEING EXCLUDED;
"

cohort <- dbGetQuery(con, cohortqry)

cohort

## Stage 2: Analyze metrics across multiple quarters pre- and post-completion and display in a line chart

We will next take a look at how average earnings of registered apprencticeship completers vary over time both before and after the quarter of completion.

This can be used to support a variety of purposes, including:
- Understanding trends in outcomes over time before and after an event or intervention of interest (i.e. apprenticeship completion)
- Understanding how trends in outcomes vary by subpopulations within your cohort (by gender, by industry, etc.)
- Producing line charts for your project presentation and report.

To complete this stage and use this query in your project:
1. Replace the **INSERT_TABLE_NAME** table name in the flower box with the name of the cohort table that you created in `02_Creating_a_cohort_checkpoints.ipynb`.  You can alternately identify your cohort by adding `WHERE` clauses to the **AR_MDIM_Person table**.
2. Analyze the results in the line chart below and explore any interesting patterns on which you may wish to conduct more detailed analyses.
3. Comment or uncomment out additional columns in the second line chart query (in both the `SELECT` and the `GROUP BY`) to slice your cohort by different variables at different grains.
4. Try different cohort sizes as well to hone in on a cohort large enough to facilitate robust analysis and dissemination on your variables of interest.

In [ ]:
# create underlying table for entire cohort of average wages received from the primary employer before and after apprenticeship completion
trendqry <- "
--DECLARE VARIABLES FOR OUR ANALYSIS
DECLARE
@PreQuarters SMALLINT,
@PostQuarters SMALLINT

--SET QUARTERS OF INTEREST
SET @PreQuarters = 8 --SET DESIRED OUTCOME QUARTERS PRECEDING THE PERIOD OF INTEREST
SET @PostQuarters = 8 --SET DESIRED OUTCOMES QUARTERS FOLLOWING THE PERIOD OF INTEREST

SELECT
F.Quarter_ID - P.Apprenticeship_End_Quarter_ID AS Quarters_Relative_to_Completion,
--COUNT(P.Person_ID) AS Completers,
ROUND(AVG(F.Primary_Employer_Wages),0) AS Avg_Primary_Employer_Wages

--,P.Gender
--,P.Race
--,P.Ethnicity
--,P.Disabled
--,P.Veteran
--,NNI.NAICS_Sector_Name
--,PEC.Rural_Urban_Continuum_Name

FROM 
--*******************************************************************************************************
--ADD YOUR TEAM'S COHORT TABLE HERE
tr_ar_2022.dbo.INSERT_TABLE C --COHORT
--AN INNER JOIN ON THE COHORT TABLE LIMITS THE SELECTION OF PERSON RECORDS TO THE COHORT OF INTEREST
--*******************************************************************************************************

JOIN tr_ar_2022.dbo.AR_MDIM_Person P ON (P.Apprentice_Number=C.apprnumber) --PERSON

JOIN tr_ar_2022.dbo.AR_FACT_Quarterly_Observation F --QUARTERLY OBSERVATION FACT
	ON (P.Person_ID=F.Person_ID) 
	AND (F.Quarter_ID BETWEEN (P.Apprenticeship_End_Quarter_ID-@PreQuarters) AND (P.Apprenticeship_End_Quarter_ID+@PostQuarters))  --QTRS PRE/POST COMPLETION
JOIN tr_ar_2022.dbo.AR_RDIM_NAICS_National_Industry NNI ON (P.Apprenticeship_NAICS_National_Industry_ID=NNI.NAICS_National_Industry_ID) --APPRENTICESHIP INDUSTRY
JOIN tr_ar_2022.dbo.AR_MDIM_Employer PE ON (PE.Employer_ID=F.Primary_Employer_ID)  --PRIMARY EMPLOYER
JOIN tr_ar_2022.dbo.AR_RDIM_County PEC ON (PEC.County_ID=PE.County_ID)  --PRIMARY EMPLOYER COUNTY

	
WHERE
P.Apprenticeship_Completer='Y'  --APPRENTICESHIP END COVERS MULTIPLE EXIT TYPES; SELECT ONLY COMPLETIONS

GROUP BY
(F.Quarter_ID - P.Apprenticeship_End_Quarter_ID) --QUARTERS_RELATIVE_TO_COMPLETION

--,P.Gender
--,P.Race
--,P.Ethnicity
--,P.Disabled
--,P.Veteran
--,NNI.NAICS_Sector_Name
--,NNI.NAICS_Subsector_Name
--,PEC.Rural_Urban_Continuum_Name

ORDER BY
(F.Quarter_ID - P.Apprenticeship_End_Quarter_ID) --QUARTERS RELATIVE TO COMPLETION
;"

trenddata <- dbGetQuery(con, trendqry)

trenddata   


In [ ]:
# slice line chart by one or more dimensions to see differences in outcomes across various subpopulations
# query to pull data in from SQL for a line chart of average earnings at primary employer by gender
trendqry2 <- "
--DECLARE VARIABLES FOR OUR ANALYSIS
DECLARE
@PreQuarters SMALLINT,
@PostQuarters SMALLINT

--SET QUARTERS OF INTEREST
SET @PreQuarters = 8 --SET DESIRED OUTCOME QUARTERS PRECEDING THE PERIOD OF INTEREST
SET @PostQuarters = 8 --SET DESIRED OUTCOMES QUARTERS FOLLOWING THE PERIOD OF INTEREST


SELECT
F.Quarter_ID - P.Apprenticeship_End_Quarter_ID AS Quarters_Relative_to_Completion,
--COUNT(P.Person_ID) AS Completers,
ROUND(AVG(F.Primary_Employer_Wages),0) AS Avg_Primary_Employer_Wages

,P.Gender
--,P.Race
--,P.Ethnicity
--,P.Disabled
--,P.Veteran
--,NNI.NAICS_Sector_Name
--,PEC.Rural_Urban_Continuum_Name

FROM 
--*******************************************************************************************************
--ADD YOUR TEAM'S COHORT TABLE HERE
tr_ar_2022.dbo.INSERT_TABLE C --COHORT
--AN INNER JOIN ON THE COHORT TABLE LIMITS THE SELECTION OF PERSON RECORDS TO THE COHORT OF INTEREST
--*******************************************************************************************************

JOIN tr_ar_2022.dbo.AR_MDIM_Person P ON (P.Apprentice_Number=C.apprnumber) --PERSON

JOIN tr_ar_2022.dbo.AR_FACT_Quarterly_Observation F --QUARTERLY OBSERVATION FACT
	ON (P.Person_ID=F.Person_ID) 
	AND (F.Quarter_ID BETWEEN (P.Apprenticeship_End_Quarter_ID-@PreQuarters) AND (P.Apprenticeship_End_Quarter_ID+@PostQuarters))  --8 QTRS PRE/POST COMPLETION
JOIN tr_ar_2022.dbo.AR_RDIM_NAICS_National_Industry NNI ON (P.Apprenticeship_NAICS_National_Industry_ID=NNI.NAICS_National_Industry_ID) --APPRENTICESHIP INDUSTRY
JOIN tr_ar_2022.dbo.AR_MDIM_Employer PE ON (PE.Employer_ID=F.Primary_Employer_ID)  --PRIMARY EMPLOYER
JOIN tr_ar_2022.dbo.AR_RDIM_County PEC ON (PEC.County_ID=PE.County_ID)  --PRIMARY EMPLOYER COUNTY

	
WHERE
P.Apprenticeship_Completer='Y'  --APPRENTICESHIP END COVERS MULTIPLE EXIT TYPES; SELECT ONLY COMPLETIONS

GROUP BY
(F.Quarter_ID - P.Apprenticeship_End_Quarter_ID) --QUARTERS_RELATIVE_TO_COMPLETION

,P.Gender
--,P.Race
--,P.Ethnicity
--,P.Disabled
--,P.Veteran
--,NNI.NAICS_Sector_Name
--,NNI.NAICS_Subsector_Name
--,PEC.Rural_Urban_Continuum_Name

ORDER BY
(F.Quarter_ID - P.Apprenticeship_End_Quarter_ID) --QUARTERS RELATIVE TO COMPLETION
;"

trenddata2 <- dbGetQuery(con, trendqry2)

trenddata2




You can then take this data and feed it directly into a line plot.

In [ ]:
# visualize overall trends in a line chart
# setting gender subgroup to the color attribute, can change based on subgroup of interest
trenddata2 %>% 
    ggplot(aes(x = Quarters_Relative_to_Completion, y = Avg_Primary_Employer_Wages, color = Gender)) +
    geom_line()

## Stage 3: Compare metrics between two periods and display in a bar chart

In this stage, we will look at two specific period of interest, such as quarter of completion and a set number of quarters following completion.  Unlike the previous analysis, we will not be pulling in every quarter, only the specific quarters of interest.  This is useful for providing a more concise and consistent snapshot for presenting outcomes and differences in outcomes across groups.

This can be used to support a variety of purposes, including:
- Calculating changes in earnings at set relative intervals post completion with the ability to include completers from multiple quarters
- Detecting changes in values at two specific periods, such as at completion and 8 quarters post-completion
- Representing changes succinctly as percent changes

To complete this stage and use this query in your project:
1. Replace the **INSERT_TABLE_NAME** table name in the flower box with the name of the cohort table that you created in `02_Creating_a_cohort_checkpoints.ipynb`.  You can alternately identify your cohort by adding `WHERE` clauses to the **AR_MDIM_Person table**.
2. Set your desired quarters of interest before and after completion. (Currently completion quarter and 8 quarters after completion)
3. Analyze the results in the bar chart below and explore any interesting patterns on which you may wish to conduct more detailed analyses.
4. Adjust the columns in the last `SELECT` and `GROUP BY` statements to slice the bar chart by a different column.  The underlying queries are designed to be very versatile for project use.

Please note when preparing export requests that underlying numbers need to be suppressed even when only exporting percentages without counts - more on this in later content.

Ensure that you don't select more quarters before or after completion than exist in your cohort (such as 8 quarters past completion for completions through CY2021).

> Note: A percentage of 100 is represented as 1.0 instead of just 1.  This is because SQL Server will allocate internal memory space based on the initial data type used.  Using 1 will allocate an integer and only return averages of 1 or 0.  1.0, though, allocates a floating point number, which will return more precise percentages.

In [ ]:
# construct table of outcomes data 
# employment retention: if employed in both quarters of interest 
# employer retention: if same primary employer in both quarters of interest
# naics supersector retention: if same naics code in apprenticeship as primary employer in second quarter of interest
# primary employer percent difference: percentage difference in earnings from primary employer between quarters of interest
barqry <- "
--DECLARE VARIABLES FOR OUR ANALYSIS
DECLARE
@PreQuarters SMALLINT,
@PostQuarters SMALLINT

--SET THE NUMBER OF QUARTERS TO PULL
SET @PreQuarters = 0 --SET DESIRED OUTCOME QUARTERS PRECEDING THE PERIOD OF INTEREST
SET @PostQuarters = 8 --SET DESIRED OUTCOMES QUARTERS FOLLOWING THE PERIOD OF INTEREST
;
--DECLARE COMMON TABLE EXPRESSION TO HOLD OBSERVATIONS ON INTEREST FOR OUR COHORT AND SELECTED QUARTERS
WITH OBSERVATIONS_OF_INTEREST AS (
	SELECT
	F.Quarter_ID - P.Apprenticeship_End_Quarter_ID AS Quarters_Relative_to_Completion
	,P.Person_ID
	,F.Primary_Employer_ID
	,F.Employed
	,F.Primary_Employer_Wages
	,F.Total_Wages
	
	--PERSON DEMOGRAPHICS
	,P.Gender
	,P.Race
	,P.Ethnicity
	,P.Disabled
	,P.Veteran
	
	--PUT APPRENTICESHIP STARTING AGE INTO STANDARD DEPT OF LABOR AGE RANGES FOR FEDERAL REPORTING
	,CASE WHEN P.Apprenticeship_Age_at_Start < 16 THEN '< 16'	
		WHEN P.Apprenticeship_Age_at_Start BETWEEN 16 AND 18 THEN '16 - 18'
		WHEN P.Apprenticeship_Age_at_Start BETWEEN 19 AND 24 THEN '19 - 24'
		WHEN P.Apprenticeship_Age_at_Start BETWEEN 25 AND 44 THEN '25 - 44'
		WHEN P.Apprenticeship_Age_at_Start BETWEEN 45 AND 54 THEN '45 - 54'
		WHEN P.Apprenticeship_Age_at_Start BETWEEN 55 AND 59 THEN '55 - 59'
		WHEN P.Apprenticeship_Age_at_Start > 60 THEN '60+'
		ELSE NULL
	END AS 'Apprenticeship_Age_Range_at_Start'
	
	--APPRENTICESHIP INDUSTRY
	,ANNI.NAICS_Supersector_Name AS Apprenticeship_NAICS_Supersector_Name
	,ANNI.NAICS_Sector_Name AS Apprenticeship_NAICS_Sector_Name
	,ANNI.NAICS_Subsector_Name AS Apprenticeship_NAICS_Subsector_Name
	,PENNI.NAICS_Industry_Group_Name AS Apprenticeship_NAICS_Industry_Group_Name
	,ANNI.NAICS_Industry_Name AS Apprenticeship_NAICS_Industry_Name
	,ANNI.NAICS_National_Industry_Name AS Apprenticeship_NAICS_National_Industry_Name
	
	--PRIMARY EMPLOYER INDUSTRY
	,PENNI.NAICS_Supersector_Name AS Primary_Employment_NAICS_Supersector_Name
	,PENNI.NAICS_Sector_Name AS Primary_Employment_NAICS_Sector_Name
	,PENNI.NAICS_Subsector_Name AS Primary_Employment_NAICS_Subsector_Name
	,PENNI.NAICS_Industry_Group_Name AS Primary_Employment_NAICS_Industry_Group_Name
	,PENNI.NAICS_Industry_Name AS Primary_Employment_NAICS_Industry_Name
	,PENNI.NAICS_National_Industry_Name AS Primary_Employment_NAICS_National_Industry_Name
	
	--APPRENTICESHIP OCCUPATION
	,S.SOC_Major_Group_Name
	,S.SOC_Minor_Group_Name
	,S.SOC_Broad_Group_Name
	,S.SOC_Detailed_Occupation_Name
	
	--APPRENTICESHIP EMPLOYER COUNTYI 
	,ACTY.Rural_Urban_Continuum_Name AS Apprenticeship_Rural_Urban_Continuum_Name
	,ACTY.Local_Workforce_Development_Area AS Apprenticeship_Local_Workforce_Development_Area
	,ACTY.County_Name AS Apprenticeship_County_Name
	
	--PRIMARY EMPLOYER COUNTYI 
	,PECTY.Rural_Urban_Continuum_Name AS Primary_Employer_Rural_Urban_Continuum_Name
	,PECTY.Local_Workforce_Development_Area AS Primary_Employer_Local_Workforce_Development_Area
	,PECTY.County_Name AS Primary_Employer_County_Name
	
	FROM 
	--*******************************************************************************************************
	--ADD YOUR TEAM'S COHORT TABLE HERE
	tr_ar_2022.dbo.INSERT_TABLE C --COHORT
	--AN INNER JOIN ON THE COHORT TABLE LIMITS THE SELECTION OF PERSON RECORDS TO THE COHORT OF INTEREST
	--*******************************************************************************************************
	
	JOIN tr_ar_2022.dbo.AR_MDIM_Person P ON (P.Apprentice_Number=C.apprnumber) --PERSON
	
	JOIN tr_ar_2022.dbo.AR_FACT_Quarterly_Observation F --QUARTERLY OBSERVATION FACT
		ON ((P.Person_ID=F.Person_ID) AND (F.Quarter_ID = (P.Apprenticeship_End_Quarter_ID-@PreQuarters))) 
		OR ((P.Person_ID=F.Person_ID) AND (F.Quarter_ID = (P.Apprenticeship_End_Quarter_ID+@PostQuarters)))
			
	--LOOK UP APPRENTICESHIP INDUSTRY WHERE APPLICABLE.  OUTER JOIN IS USED TO AVOID EXCLUDING ROWS WITHOUT INDUSTRY.
	LEFT JOIN tr_ar_2022.dbo.AR_RDIM_NAICS_National_Industry ANNI ON (ANNI.NAICS_National_Industry_ID=P.Apprenticeship_NAICS_National_Industry_ID)
	
	--LOOK UP APPRENTICESHIP OCCUPATION WHERE APPLICABLE.  OUTER JOIN IS USED TO AVOID EXCLUDING ROWS WITHOUT OCCUPATION.
	LEFT JOIN tr_ar_2022.dbo.AR_RDIM_SOC_Detailed_Occupation S ON (S.SOC_Detailed_Occupation_ID=P.Apprenticeship_SOC_Detailed_Occupation_ID)
	
	--LOOK UP APPRENTICESHIP PROGRAM COUNTY WHERE APPLICABLE.  OUTER JOIN IS USED TO AVOID EXCLUDING ROWS WITHOUT COUNTY.
	LEFT JOIN tr_ar_2022.dbo.AR_RDIM_County ACTY ON (ACTY.County_ID=P.Apprenticeship_County_ID)	
	
	--LOOK UP PRIMARY EMPLOYER WHERE APPLICABLE
	LEFT JOIN tr_ar_2022.dbo.AR_MDIM_Employer PE ON (PE.Employer_ID=F.Primary_Employer_ID)  --PRIMARY EMPLOYER
	
	--LOOK UP PRIMARY EMPLOYER COUNTY WHERE APPLICABLE
	LEFT JOIN tr_ar_2022.dbo.AR_RDIM_County PECTY ON (PECTY.County_ID=PE.County_ID)  --PRIMARY EMPLOYER COUNTY
	
	--LOOK UP PRIMARY EMPLOYER INDUSTRY WHERE APPLICABLE
	LEFT JOIN tr_ar_2022.dbo.AR_RDIM_NAICS_National_Industry PENNI ON (PE.NAICS_National_Industry_ID=PENNI.NAICS_National_Industry_ID) --APPRENTICESHIP INDUSTRY
	
	WHERE
	P.Apprenticeship_Completer='Y'  --APPRENTICESHIP END COVERS MULTIPLE EXIT TYPES; SELECT ONLY COMPLETIONS
)
--CALCULATE OUR PERSON-LEVEL LONGITUDINAL OUTCOMES
,PERSON_LONGITUDINAL_OUTCOMES AS (
	SELECT
	--PERSON CHARACTERISTICS
	FO.Person_ID
	,FO.Gender
	,FO.Race
	,FO.Ethnicity
	,FO.Disabled
	,FO.Veteran
	,FO.Apprenticeship_Age_Range_at_Start

	--EMPLOYER
	,FO.Employed AS Employed_First_Observation
	,SO.Employed AS Employed_Second_Observation
	,CASE WHEN FO.Employed = 'Y' AND SO.Employed = 'Y' THEN 1.0 ELSE 0 END AS Employment_Retention

	,FO.Primary_Employer_ID AS Primary_Employer_ID_First_Observation
	,SO.Primary_Employer_ID AS Primary_Employer_ID_Second_Observation
	,CASE WHEN FO.Primary_Employer_ID=SO.Primary_Employer_ID THEN 1.0 ELSE 0 END AS Same_Primary_Employer_Retention

	,FO.Primary_Employer_Wages AS Primary_Employer_Wages_First_Observation
	,SO.Primary_Employer_Wages AS Primary_Employer_Wages_Second_Observation
	,CASE WHEN FO.Employed = 'Y' AND SO.Employed = 'Y' THEN SO.Primary_Employer_Wages - FO.Primary_Employer_Wages ELSE 0 END AS Primary_Employer_Wages_Difference
	,CASE WHEN FO.Employed = 'Y' AND SO.Employed = 'Y' THEN (SO.Primary_Employer_Wages - FO.Primary_Employer_Wages) / FO.Primary_Employer_Wages ELSE 0 END AS Primary_Employer_Wages_Percent_Difference

	,FO.Total_Wages AS Total_Wages_First_Observation
	,SO.Total_Wages AS Total_Wages_Second_Observation


	--INDUSTRY
	,FO.Apprenticeship_NAICS_Supersector_Name
	,SO.Primary_Employment_NAICS_Supersector_Name
	,CASE WHEN FO.Apprenticeship_NAICS_Supersector_Name = SO.Primary_Employment_NAICS_Supersector_Name THEN 1.0 ELSE 0 END AS Same_NAICS_Supersector_Retention

	,FO.Apprenticeship_NAICS_Sector_Name
	,SO.Primary_Employment_NAICS_Sector_Name
	,FO.Apprenticeship_NAICS_Subsector_Name
	,SO.Primary_Employment_NAICS_Subsector_Name
	,FO.Apprenticeship_NAICS_Industry_Group_Name
	,SO.Primary_Employment_NAICS_Industry_Group_Name
	,FO.Apprenticeship_NAICS_Industry_Name
	,SO.Primary_Employment_NAICS_Industry_Name
	,FO.Apprenticeship_NAICS_National_Industry_Name
	,SO.Primary_Employment_NAICS_National_Industry_Name

	--COUNTY
	,FO.Apprenticeship_County_Name
	,SO.Primary_Employer_County_Name
	,FO.Apprenticeship_Local_Workforce_Development_Area
	,SO.Primary_Employer_Local_Workforce_Development_Area
	,FO.Apprenticeship_Rural_Urban_Continuum_Name
	,SO.Primary_Employer_Rural_Urban_Continuum_Name

	--OCCUPATION
	,FO.SOC_Major_Group_Name
	,FO.SOC_Minor_Group_Name
	,FO.SOC_Broad_Group_Name
	,FO.SOC_Detailed_Occupation_Name

	FROM 
	OBSERVATIONS_OF_INTEREST FO --FIRST OBSERVATION
	JOIN OBSERVATIONS_OF_INTEREST SO --SECOND OBSERVATION
		ON  (FO.Person_ID=SO.Person_ID)
	WHERE
		FO.Quarters_Relative_to_Completion=@PreQuarters
		AND SO.Quarters_Relative_to_Completion=@PostQuarters
)

SELECT
PLO.Apprenticeship_NAICS_SuperSector_Name
,AVG(PLO.Employment_Retention) AS Employment_Retention_Rate
,AVG(PLO.Same_Primary_Employer_Retention) AS Primary_Employer_Retention_Rate
,AVG(PLO.Same_NAICS_Supersector_Retention) AS NAICS_Supersector_Retention_Rate
,AVG(PLO.Primary_Employer_Wages_Percent_Difference) AS Primary_Employer_Wages_Percent_Difference

FROM
PERSON_LONGITUDINAL_OUTCOMES PLO

GROUP BY
PLO.Apprenticeship_NAICS_SuperSector_Name
;"

bardata <- dbGetQuery(con, barqry)

head(bardata)


In [ ]:
# visualize data in a bar chart here to highlight the differences in the metrics above sliced by industry
# requires making the data "long" from "wide" so that one row is a metric/industry combination
# include -Apprenticeship_NAICS_SuperSector_Name in pivot_longer() so that it's a separate column
# facet_wrap will create panels by a specific subgroup (measure)--more to come on visualization methods in ggplot
bardata %>% 
    pivot_longer(names_to = "measure", values_to = "proportion", -Apprenticeship_NAICS_SuperSector_Name) %>% 
    ggplot(aes(x = Apprenticeship_NAICS_SuperSector_Name,  y = proportion)) + 
    geom_col() +
    facet_wrap(~measure) + 
    scale_y_continuous(labels = scales::percent) # the scales are all percentages